## Settings

In [83]:
# 一つ上の階層からモジュールを参照できるようにする
import sys
sys.path.append('..')

In [84]:
# モジュールの変更を自動的に反映する
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [85]:
import numpy as np
import pandas as pd
from inputs import load_data
# XGB
from models import XGB
# Torch
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from models import TrainingDataset, Torch, transform_labels, restore_labels, train_model, validate_model, \
                   TestDataset, predict_test
# Keras
from keras import Sequential, Input
from keras.utils import to_categorical
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

## XGB

In [15]:
train_x, train_y, test_x = load_data()

In [8]:
params = None
model = XGB(params)

In [9]:
tr_x = train_x#[:80]
va_x = train_x#[80:]
tr_y = train_y#[:80]
va_y = train_y#[80:]

In [15]:
early_stopping = 5
model.fit(tr_x, tr_y, va_x, va_y,
          early_stopping_rounds=early_stopping,
          verbose=False)

In [11]:
preds = model.predict(test_x)

In [12]:
index = test_x.index.to_numpy().reshape(-1, 1)
preds = preds.reshape(-1, 1)
submission = np.concatenate((index, preds), axis=1)
submission = pd.DataFrame(submission)

In [13]:
submission.to_csv('xgb.csv', index=False, header=False)

## Torch

In [251]:
dataset = TrainingDataset(target_transform = transform_labels)

In [252]:
train_ratio = 0.95
train_size = int(train_ratio * len(dataset))
test_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, test_size])

In [253]:
batch_size = 10

train_dataloader = DataLoader(train_dataset, batch_size=batch_size)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

for X, y in val_dataloader:
    print('Shape of X', X.shape)
    print('Shape of y', y.shape, y.dtype)
    break

Shape of X torch.Size([6, 9])
Shape of y torch.Size([6]) torch.int64


In [254]:
input_size = 9
output_size = 6
model = Torch(input_size, output_size)

In [255]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [256]:
epochs = 5
for t in range(epochs):
    print(f'Epoch {t+1}\n-------------------------------')
    train_model(train_dataloader, model, loss_fn, optimizer)
    validate_model(val_dataloader, model, loss_fn)
print('Done!')

Epoch 1
-------------------------------
loss: 1.652944  [    0/  101]
loss: 1.832621  [   20/  101]
loss: 1.747383  [   40/  101]
loss: 1.552773  [   60/  101]
loss: 1.837954  [   80/  101]
loss: 2.033239  [  100/  101]
Test Error: 
Accuracy: 50.0%, Avg loss: 0.259492 

Epoch 2
-------------------------------
loss: 1.652390  [    0/  101]
loss: 1.833190  [   20/  101]
loss: 1.747138  [   40/  101]
loss: 1.552252  [   60/  101]
loss: 1.838223  [   80/  101]
loss: 2.033751  [  100/  101]
Test Error: 
Accuracy: 50.0%, Avg loss: 0.259368 

Epoch 3
-------------------------------
loss: 1.651894  [    0/  101]
loss: 1.833707  [   20/  101]
loss: 1.746919  [   40/  101]
loss: 1.551783  [   60/  101]
loss: 1.838471  [   80/  101]
loss: 2.034219  [  100/  101]
Test Error: 
Accuracy: 50.0%, Avg loss: 0.259256 

Epoch 4
-------------------------------
loss: 1.651446  [    0/  101]
loss: 1.834180  [   20/  101]
loss: 1.746724  [   40/  101]
loss: 1.551358  [   60/  101]
loss: 1.838698  [   80/  10

In [257]:
test_dataset = TestDataset()
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)
preds = predict_test(test_dataloader, model)

In [258]:
preds = [restore_labels(p) for p in preds]
preds = np.array(preds)

In [260]:
# preds

In [261]:
_, _, test_x = load_data()
index = test_x.index.to_numpy().reshape(-1, 1)
preds = preds.reshape(-1, 1)
submission = np.concatenate((index, preds), axis=1)
submission = pd.DataFrame(submission)

In [262]:
submission.to_csv('torch.csv', index=False, header=False)

## Keras

In [212]:
TRAINING_FOR_SUBMISSION = True
validation_split = 0.0 if TRAINING_FOR_SUBMISSION else 0.4

In [213]:
train_x, train_y, test_x = load_data()

In [214]:
num_classes = len(train_y.unique())
train_y = pd.get_dummies(train_y)

In [215]:
input_shape = (train_x.shape[1],)
model = Sequential([
    Input(shape=input_shape),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(num_classes, activation='softmax')
])
model.summary()

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_66 (Dense)             (None, 64)                640       
_________________________________________________________________
dense_67 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_68 (Dense)             (None, 6)                 390       
Total params: 5,190
Trainable params: 5,190
Non-trainable params: 0
_________________________________________________________________


In [216]:
learning_rate = 1e-3
optimizer = Adam(learning_rate=learning_rate)

In [217]:
batch_size = 10
epochs = 5

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, validation_split=validation_split)

Epoch 1/5
11/11 [==============================] - 0s 852us/step - loss: 8.0530 - accuracy: 0.1682
Epoch 2/5
11/11 [==============================] - 0s 789us/step - loss: 3.1567 - accuracy: 0.3551
Epoch 3/5
11/11 [==============================] - 0s 829us/step - loss: 2.0668 - accuracy: 0.3178
Epoch 4/5
11/11 [==============================] - 0s 903us/step - loss: 1.8164 - accuracy: 0.3738
Epoch 5/5
11/11 [==============================] - 0s 933us/step - loss: 1.5992 - accuracy: 0.4112


In [218]:
classes = train_y.columns.tolist()
preds = model.predict(test_x).argmax(axis=1)
preds = [classes[pred] for pred in preds]
preds = np.array(preds)
preds = preds.reshape(-1, 1)

In [222]:
# preds

In [220]:
index = test_x.index.to_numpy().reshape(-1, 1)
submission = np.concatenate((index, preds), axis=1)
submission = pd.DataFrame(submission)

In [221]:
submission.to_csv('keras.csv', index=False, header=False)